In [1]:
import datetime
import pandas as pd
import mysql_adactivity_save

Using TensorFlow backend.


In [ ]:
import tqdm

In [2]:
DATADASE = "ad_activity"
START_TIME = 'start_time'
STOP_TIME = 'stop_time'
AD_ID = 'ad_id'
ADSET_ID = 'adset_id'
CAMPAIGN_ID = 'campaign_id'
CHARGE = 'charge'
BID_AMOUNT = 'bid_amount'
REQUEST_TIME = 'request_time'
TARGET_LEFT = 'target_left'

In [229]:
class FacebookCampaignAdapter(object):
    def __init__(self, campaign_id):
        self.mydb = mysql_adactivity_save.connectDB( DATADASE )
        self.limit = 10000
        self.hour_per_day = 20
        self.campaign_id = campaign_id
        self.request_time = datetime.datetime.now()
        self.time_progress = ( self.request_time.hour + 1 ) / self.hour_per_day
        
    def get_df(self):
        self.df_camp = pd.read_sql( "SELECT * FROM campaign_target WHERE campaign_id=%s" %( self.campaign_id ), con=self.mydb )
        self.df_ad = pd.read_sql( "SELECT * FROM ad_insights where campaign_id = %s ORDER BY request_time DESC LIMIT %s" %( self.campaign_id, self.limit ), con=self.mydb )
        
        adset_list = self.get_adset_list()
        sql = "SELECT adset_id, bid_amount, request_time FROM adset_insights WHERE adset_id IN (%s);" % (','.join(['%s'] * len(adset_list)) )
        val = adset_list
        
        self.df_adset = pd.read_sql( sql, con=self.mydb, params = adset_list )
        return
    
    def get_campaign_days_left(self):
        self.campaign_days_left = ( self.df_camp[ STOP_TIME ].iloc[0] - self.request_time ).days
        return self.campaign_days_left
    
    def get_campaign_days(self):
        self.campaign_days = ( self.df_camp[ STOP_TIME ].iloc[0] - self.df_camp[ START_TIME ].iloc[0] ).days
        return self.campaign_days
    
    def get_campaign_performance(self):
#         df_ads = pd.read_sql( "SELECT * FROM ad_insights where campaign_id = %s ORDER BY request_time desc limit %s" %( self.campaign_id, self.limit ), con=self.mydb )
        ad_id_list = self.df_ad[ AD_ID ].unique()
        dfs = pd.DataFrame(columns=[ ADSET_ID, CHARGE ])
#         for ad_id in tqdm(ad_id_list):
        for ad_id in ad_id_list:
            ad_id = ad_id.astype(dtype=object)
            df_ad = self.df_ad[self.df_ad.ad_id==ad_id]
            adset_id = df_ad[ ADSET_ID ].iloc[0]
            df = df_ad[[ CHARGE, REQUEST_TIME ]][df_ad.request_time.dt.date==self.request_time]
            dfs = pd.concat([dfs, df], axis=0, sort=False)
        dfs = dfs.sort_values(by=[ CHARGE ], ascending=False).reset_index(drop=True)
        self.campaign_performance = dfs[ CHARGE ].sum()
        return self.campaign_performance
    
    def get_campaign_target(self):
        self.campaign_target = self.df_camp[ TARGET_LEFT ].iloc[0].astype(dtype=object)
        return self.campaign_target
    
    def get_campaign_day_target(self):
        self.campaign_day_target = self.campaign_target / self.campaign_days_left
        return self.campaign_day_target
    
    def get_adset_list(self):
        df_ad = pd.read_sql( "SELECT * FROM ad_insights where campaign_id = %s ORDER BY request_time desc limit %s" %( self.campaign_id, self.limit ), con=self.mydb )
        return df_ad[ ADSET_ID ].unique().tolist()
    
    def retrieve_campaign_attribute(self):
        self.get_df()
#         self.get_campaign_days_left()
#         self.get_campaign_days()
#         self.get_campaign_performance()
#         self.get_campaign_target()
#         self.get_campaign_day_target()
        return

In [228]:
%%time
fb = FacebookCampaignAdapter(23843355587140564)
fb.retrieve_campaign_attribute()

            adset_id  bid_amount
0  23843355587240564          11
1  23843355587250564          11
2  23843355587240564          11
3  23843355587250564          13
4  23843355587240564          11
CPU times: user 2.28 s, sys: 304 ms, total: 2.58 s
Wall time: 3.2 s


In [159]:
class FacebookAdSetAdapter(FacebookCampaignAdapter):
    def __init__(self, adset_id, fb):
        self.mydb = mysql_adactivity_save.connectDB( DATADASE )
        self.adset_id = adset_id
        self.fb = fb
#         self.df_adset = pd.read_sql( "SELECT * FROM adset_insights WHERE adset_id=%s ORDER BY request_time DESC LIMIT 1" %( self.adset_id ), con=self.mydb )
#         self.df_adset = pd.read_sql( "SELECT * FROM adset_insights WHERE adset_id=%s" %( self.adset_id ), con=self.mydb )

    def init_campaign(self, fb):
        self.time_progress = fb.time_progress
        self.limit = fb.limit
        self.hour_per_day = fb.hour_per_day
        self.request_time = fb.request_time
        self.df_camp = fb.df_camp
        self.df_ad = fb.df_ad
        self.df_adset = fb.df_adset
        self.campaign_days_left = fb.campaign_days_left
        self.campaign_days = fb.campaign_days
        self.campaign_performance = fb.campaign_performance
        self.campaign_target = fb.campaign_target
        self.campaign_day_target = fb.campaign_day_target
        return
    
    def get_campaign_id(self):
        df_ad = pd.read_sql( "SELECT * FROM ad_insights WHERE adset_id = %s ORDER BY request_time DESC LIMIT 1" %( self.adset_id ), con=self.mydb )
        self.campaign_id = df_ad[ CAMPAIGN_ID ].iloc[0].astype(dtype=object)
        return self.campaign_id
    
    def get_adset_day_target(self):
        adset_num = len( self.df_ad[ ADSET_ID ].unique() )
        self.adset_day_target = self.fb.campaign_day_target / adset_num
        return self.adset_day_target
    
    def get_adset_performance(self):
        self.adset_performance = self.df_ad[ CHARGE ][self.df_ad.adset_id==self.adset_id].iloc[0]
        return self.adset_performance
    
    def get_init_bid(self):
#         df_adset = pd.read_sql( "SELECT * FROM adset_insights WHERE adset_id=%s LIMIT 1" %( self.adset_id ), con=self.mydb )
        self.init_bid = self.df_adset[ BID_AMOUNT ].iloc[0]
        return self.init_bid
    
    def get_last_bid(self):
        self.df_adset
        self.last
        
        
        
        mycursor = self.mydb.cursor()
#         sql = "SELECT bid_amount FROM adset_insights WHERE adset_id=%s ORDER BY request_time DESC LIMIT 1"
#         mycursor.execute("SELECT bid_amount FROM adset_insights WHERE adset_id=%s ORDER BY request_time DESC LIMIT 1" % (self.adset_id) )
        mycursor.execute("SELECT bid_amount FROM adset_insights WHERE adset_id=%s" % (self.adset_id) )
        last_bid = mycursor.fetchall()
#         print(last_bid[0][0])
#         df_adset = pd.read_sql( "SELECT * FROM adset_insights WHERE adset_id=%s ORDER BY request_time DESC LIMIT 1" %( self.adset_id ), con=self.mydb )
#         df_adset = pd.read_sql( "SELECT * FROM adset_insights WHERE adset_id=%s" %( self.adset_id ), con=self.mydb )
#         self.last_bid = df_adset[ BID_AMOUNT ].tail(1).iloc[0].astype(dtype=object)
    
#         self.last_bid = self.df_adset[ BID_AMOUNT ].tail(1).iloc[0].astype(dtype=object)
        return #self.last_bid
    
    def get_adset_time_target(self):
        self.adset_time_target = self.adset_day_target * self.time_progress
        return self.adset_time_target
    
    def get_adset_progress(self):
        self.adset_progress = self.adset_performance / self.adset_time_target
        return self.adset_progress
    
    def retrieve_adset_attribute(self):
#         self.get_campaign_id()
#         self.init_campaign(self.fb)
#         self.get_adset_day_target()
#         self.get_adset_performance()
#         self.get_init_bid()
        self.get_last_bid()###
#         self.get_adset_time_target()
#         self.get_adset_progress()
        return

In [75]:
%%time
fb = FacebookCampaignAdapter(23843355587140564)
fb.retrieve_campaign_attribute()


CPU times: user 236 ms, sys: 16 ms, total: 252 ms
Wall time: 572 ms


In [160]:
%%time
s = FacebookAdSetAdapter(23843355587230564, fb)
s.retrieve_adset_attribute()

CPU times: user 128 ms, sys: 12 ms, total: 140 ms
Wall time: 349 ms


In [113]:
%%time
camp_list = [23843355587140564]
for camp in camp_list:
    fb = FacebookCampaignAdapter(camp)
    fb.retrieve_campaign_attribute()
    adset_list = fb.get_adset_list()
    for adset in adset_list:
        s = FacebookAdSetAdapter(adset, fb)
        s.retrieve_adset_attribute()

CPU times: user 540 ms, sys: 36 ms, total: 576 ms
Wall time: 10.2 s
